## Protocole 2 : Étude des Performances de Présentation de Stimuli Visuels

Ce protocole vise à vérifier la précision et la stabilité temporelle d'apparition effective d'un stimulus visuel sur un écran.

### Prérequis

**Matériel :**
*   Un oscilloscope numérique ou un dispositif d'acquisition (ici Analog Discovery 3)
*   **Deux écrans d'ordinateur.**
*   Une **photodiode** avec son circuit d'alimentation.

**Logiciels :**
*   Logiciel d'acquisition et de mesure (ici **Digilent Waveforms**, version `3.24.2`)
*   **Python 3** avec les bibliothèques supplémentaires nécessaires (ex: `pyserial`, `expyriment`, `psychopy2`).

### Étapes à suivre

#### 1. Génération des Stimuli

1.  **Disposez de deux écrans** : un pour la fenêtre d’enregistrement (Waveforms) et l’autre pour l’affichage des stimuli visuels.

2.  **Préparez cotre code .py** qui affiche les stimuli visuels, en prenant en argument la période désirée. MODIFIER CODE

> **Code Python (.py) pour l’affichage des stimuli.**
> ```python
> from expyriment import design, control, stimuli, misc
>
> # --- Setup Expyriment ---
> exp = design.Experiment(name="Stimuli_Only")
> control.set_develop_mode(True)
> control.initialize(exp)
>
> PERIOD = 250  # Intervalle total en ms
> SQUARE_DURATION = 100  # ms
>
> square = stimuli.Rectangle((400, 400), position=(0, 0))
> blank = stimuli.BlankScreen()
> square.preload()
> blank.preload()
>
> exp.add_data_variable_names(['trial', 'stimulus_time'])
>
> control.start(skip_ready_screen=True)
> clock2 = misc.Clock()
> i = 1
>
> while i <= 10000:
>     while (clock2.time < i * PERIOD - 3):
>         pass
>
>     stim_time = clock2.time + square.present(update=True)
>
>     while (clock2.time - stim_time < SQUARE_DURATION - 3):
>         pass
>     blank.present(update=True)
>
>     exp.data.add([i, stim_time])
>     i += 1
>     exp.keyboard.process_control_keys()
>
> control.end()
> ```



3.  **Lancez le code .py** sur le terminal de votre ordinateur.
![Scope settings](../pictures/expy_stimuli_only.png)

4.  Vous devriez maintenant avoir une fenêtre présentant les stimuli. **Placez cette fenêtre sur l'écran secondaire**. 

#### 2. Mesure des Signaux

1.  **Connectez les sondes de l'oscilloscope :**
        *   Broche GND de l'oscilloscope sur le GND de la photodiode.
        *   Broche de mesure sur la sortie (Vout) de la photodiode.

2.  **Placez la photodiode** au niveau de la fenêtre où sont présentés les stimuli.

3.  **Lancez le logiciel Waveforms** (ou équivalent). Cliquez sur **`Scope`**
![Scope settings](../pictures/welcome_page_waveforms.png)
![Scope settings](../pictures/scope_opened.png)

4. Cliquez sur **`Run`** pour démarrer l'acquisition. Vous devriez observer un signal qui varie en fonction de l'intensité lumineuse présenté à la photodiode sur le canal correspondant à vos branchements.
![Scope settings](../pictures/acquisition_photodiode.png)

5.  Vous pouvez étudier à l'œil le signal (période, amplitude, rapport cyclique) en **changeant l’échelle des axes**.
![Scope settings](../pictures/acquisition_characteristics_one_channel.png)

6.  **Ajustez les échelles** et lancez l'acquisition avec le bouton **`Run`**.

#### 3. Enregistrement des Signaux

1.  Cliquez sur le bouton **`Record`**.
![Scope settings](../pictures/record_button.png)

2.  Configurez les paramètres d'enregistrement :
    *   **Format du fichier :** `.wav`
    *   **Fréquence d'échantillonnage :** `10 kHz`
    *   **Durée d’enregistrement :** `20 s`
    *   **Emplacement et nom de votre fichier.**
![Scope settings](../pictures/choice_of_recording_characteristics_wav.png)
3.  Lancez l’enregistrement. Vous avez alors un fichier `.wav` comportant le signal mesuré..

#### 4. Analyse du Signal

1.  **Préparez le script Python**. Le code doit permettre l’étude du fichier `.wav` en choisissant un seuil de détection, un intervalle minimum entre les pics, et d'autres paramètres. Il doit étudier les intervalles min, max et moyen, générer un histogramme de la répartition des intervalles, et compter le nombre de pics et d'intervalles.

    > **Code Python (.py) pour l’analyse du signal de la photodiode.**
> ```python    
> import numpy as np
> import matplotlib
> matplotlib.use('Agg')  # Backend non interactif
> import matplotlib.pyplot as plt
> from scipy.io import wavfile
> from scipy.signal import find_peaks
> import os
> 
> # --- Paramètres ---
> NOM_FICHIER_WAV = 'psychopy_stimuli_only_250ms.wav'  # Nom du fichier WAV à analyser
> DISTANCE_MIN_MS = 100.0                       # Intervalle minimal entre pics
> 
> def analyser_et_sauvegarder_graphique(nom_fichier):
>     """
>     Analyse un signal photodiode au format WAV :
>     - détection des pics
>     - calcul des intervalles
>     - histogramme des intervalles
>     - tracé du signal avec pics
>     """
>     try:
>         sampling_rate, data = wavfile.read(nom_fichier)
>         print(f"\n--- Analyse du fichier : {nom_fichier} ---")
>         print(f"Taux d'échantillonnage : {sampling_rate} Hz")
> 
>         if data.ndim > 1:
>             data = data[:, 0]
> 
>         signal_abs = np.abs(data)
>         seuil = 0.8 * np.max(signal_abs)
>         print(f"Seuil de détection (80% amplitude max) : {int(seuil)}")
> 
>         distance_min_samples = int(DISTANCE_MIN_MS * sampling_rate / 1000)
> 
>         indices_pics, _ = find_peaks(signal_abs, height=seuil, distance=distance_min_samples)
>         intervalles_en_samples = np.diff(indices_pics)
>         intervalles_en_ms = (intervalles_en_samples / sampling_rate) * 1000
> 
>         print(f"\nNombre de pics détectés : {len(indices_pics)}")
>         print(f"Nombre d'intervalles : {len(intervalles_en_ms)}")
> 
>         if len(intervalles_en_ms) > 0:
>             moyenne = np.mean(intervalles_en_ms)
>             print(f"Intervalle moyen : {moyenne:.2f} ms")
>             print(f"Min : {np.min(intervalles_en_ms):.2f} ms")
>             print(f"Max : {np.max(intervalles_en_ms):.2f} ms")
> 
>             # --- Histogramme des intervalles ---
>             plt.figure(figsize=(12, 6))
>             plt.hist(intervalles_en_ms, bins=50, edgecolor='black', alpha=0.75)
>             plt.axvline(moyenne, color='red', linestyle='--', linewidth=2, label=f'Moyenne = {moyenne:.2f} ms')
>             plt.title(f"Distribution des intervalles — {os.path.basename(nom_fichier)}", fontsize=14)
>             plt.xlabel("Durée de l'intervalle (ms)")
>             plt.ylabel("Nombre d'intervalles")
>             plt.legend()
>             plt.grid(True, linestyle='--', alpha=0.6)
> 
>             output_dir = 'figures'
>             os.makedirs(output_dir, exist_ok=True)
>             base_name = os.path.splitext(os.path.basename(nom_fichier))[0]
>             save_hist_path = os.path.join(output_dir, f'distribution_intervalles_{base_name}.png')
>             plt.savefig(save_hist_path, dpi=300, bbox_inches='tight')
>             plt.close()
>             print(f"\nHistogramme sauvegardé : {save_hist_path}")
> 
>             # --- Tracé du signal avec pics ---
>             plt.figure(figsize=(15, 5))
>             plt.plot(data, label='Signal')
>             plt.plot(indices_pics, data[indices_pics], 'rx', label='Pics détectés')
>             plt.title(f"Signal et pics détectés — {base_name}")
>             plt.xlabel("Échantillons")
>             plt.ylabel("Amplitude")
>             plt.legend()
>             save_signal_path = os.path.join(output_dir, f'signal_pics_{base_name}.png')
>             plt.savefig(save_signal_path, dpi=300, bbox_inches='tight')
>             plt.close()
>             print(f"Tracé du signal sauvegardé : {save_signal_path}")
> 
>         else:
>             print("Aucun intervalle détecté — pas de figure générée.")
> 
>     except FileNotFoundError:
>         print(f"ERREUR : Le fichier '{nom_fichier}' est introuvable.")
>     except Exception as e:
>         print(f"Une erreur est survenue : {e}")
> 
> # --- Lancement automatique ---
> if __name__ == '__main__':
>     analyser_et_sauvegarder_graphique(NOM_FICHIER_WAV)
> ```


2.  **Lancez le code**.

3.  Vous avez désormais affiché sur le terminal les caractéristiques du signal et celles des intervalles de latence, ainsi que l'histogramme correspondant que vous pouvez retrouver au chemin d'accès renseigné.---
![Scope settings](../pictures/command_results_photodiode.png)
![Scope settings](../pictures/results_plot_photodiode.png)
